In [9]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import numpy as np
import rasterio
from datetime import datetime
from tqdm import tqdm
import scipy.stats as stats
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.ndimage import uniform_filter 

In [14]:
# Directories for data
rainfall_dir = r"/home/stormej/dev/varsha/data/rain/rain_resampled_monthly"
ndvi_dir = r"/home/stormej/dev/varsha/data/ndvi/ndvi_resampled_monthly"
lst_dir = r"/home/stormej/dev/varsha/data/lst/lst_composite_delhi_monthly_resampled"
dem_path = r"/home/stormej/dev/varsha/data/dem/dem_resampled.tif"

start_year = 2000
end_year = 2022


In [15]:
import os
from tqdm import tqdm
import rasterio
import numpy as np
from datetime import datetime
from scipy import stats

def extract_date_from_filename(filename, prefix, date_format):
    """Extract date from filename based on given prefix and format"""
    date_str = filename.replace(prefix, '').replace('.tif', '')
    return datetime.strptime(date_str, '%Y-%m')

def load_and_process_data(
    rainfall_dir, 
    ndvi_dir, 
    lst_dir, 
    start_year=start_year, 
    end_year=end_year
):
    """Load and process raster data for rainfall prediction"""
    # List files
    rainfall_files = sorted(os.listdir(rainfall_dir))
    ndvi_files = sorted(os.listdir(ndvi_dir))
    lst_files = sorted(os.listdir(lst_dir))
    
    data_collection = []
    
    # Use tqdm for progress tracking
    for rf in tqdm(rainfall_files, desc="Processing files"):
        try:
            rf_date = extract_date_from_filename(rf, 'imd_rain_sum_resampled_', '%Y-%m')
            
            # Filter by year range
            if not (start_year <= rf_date.year <= end_year):
                continue
            
            # Find matching NDVI and LST files
            matching_ndvi = next((nf for nf in ndvi_files 
                                  if extract_date_from_filename(nf, 'modis_ndvi_avg_resampled_', '%Y_%m') == rf_date), None)
            matching_lst = next((lf for lf in lst_files 
                                 if extract_date_from_filename(lf, 'modis_lst_avg_resampled_', '%Y-%m') == rf_date), None)
            
            if matching_ndvi and matching_lst:
                # Load rainfall data
                with rasterio.open(os.path.join(rainfall_dir, rf)) as src:
                    rainfall_data = src.read(1)
                    rainfall_flat = rainfall_data.flatten()
                    
                    # Get coordinates for each pixel
                    shape = rainfall_data.shape
                    coords = np.array([src.xy(row, col) for row in range(shape[0]) for col in range(shape[1])])
                    latitudes, longitudes = coords[:, 1], coords[:, 0]
                
                # Load NDVI data
                with rasterio.open(os.path.join(ndvi_dir, matching_ndvi)) as src:
                    ndvi_flat = src.read(1).flatten()
                
                # Load LST data
                with rasterio.open(os.path.join(lst_dir, matching_lst)) as src:
                    lst_flat = src.read(1).flatten()
                
                # Add month as cyclic feature
                month = rf_date.month
                month_sin = np.sin(2 * np.pi * month / 12)
                month_cos = np.cos(2 * np.pi * month / 12)
                month_sin_array = np.full_like(rainfall_flat, month_sin)
                month_cos_array = np.full_like(rainfall_flat, month_cos)
                
                # Create comprehensive feature array
                feature_stack = np.column_stack([
                    ndvi_flat,           # Normalized Difference Vegetation Index
                    lst_flat,             # Land Surface Temperature
                    latitudes,            # Latitude
                    longitudes,           # Longitude
                    month_sin_array,      # Cyclic month (sine)
                    month_cos_array       # Cyclic month (cosine)
                ])
                
                # Store valid data
                mask = (~np.isnan(rainfall_flat) & 
                        ~np.isnan(feature_stack).any(axis=1))
                
                if mask.any():
                    data_collection.append({
                        'date': rf_date,
                        'features': feature_stack[mask],
                        'rainfall': rainfall_flat[mask]
                    })
        
        except Exception as e:
            print(f"Error processing {rf}: {e}")
            continue
    
    return data_collection

def remove_outliers(X, y, n_sigma=3):
    """Remove outliers using z-score method"""
    z_scores = np.abs(stats.zscore(X, nan_policy='omit'))
    mask = (z_scores < n_sigma).all(axis=1)
    return X[mask], y[mask]

# Load data
print("Starting data loading...")
data_collection = load_and_process_data(
    rainfall_dir, 
    ndvi_dir, 
    lst_dir
)

Starting data loading...


Processing files: 100%|██████████| 276/276 [01:07<00:00,  4.11it/s]


In [16]:
# Combine all data

data_collection = load_and_process_data(
    rainfall_dir, 
    ndvi_dir, 
    lst_dir
)

X = np.vstack([d['features'] for d in data_collection])
y = np.concatenate([d['rainfall'] for d in data_collection])

# Remove outliers
rainfall_mask = (y >= 0) & (y < np.percentile(y, 99.9))
feature_mask = np.all((X > np.percentile(X, 0.1, axis=0)) & 
                      (X < np.percentile(X, 99.9, axis=0)), axis=1)
mask = rainfall_mask & feature_mask
X_cleaned = X[mask]
y_cleaned = y[mask]

# Remove statistical outliers
X_cleaned, y_cleaned = remove_outliers(X_cleaned, y_cleaned)

# Scale features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# Split data (temporal split)
split_idx = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_test = y_cleaned[:split_idx], y_cleaned[split_idx:]

Processing files: 100%|██████████| 276/276 [01:07<00:00,  4.09it/s]


In [17]:
from sklearn.model_selection import GridSearchCV

# Define and train Random Forest model with reduced hyperparameter tuning
# Define the parameter grid
param_grid = {
    'n_estimators': [200],
    'max_depth': [5],
    'min_samples_split': [12],
    'min_samples_leaf': [4],
    'max_features': ['sqrt']
}

# Initialize the model
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Define the model with the best parameters
model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    random_state=42,
    n_jobs=-1,
    verbose=1
)

print("\nTraining model...")
model.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=  40.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=  40.5s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=12, n_estimators=200; total time=  40.8s
Best parameters found: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 12, 'n_estimators': 200}

Training model...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   21.0s finished


RandomForestRegressor(max_depth=5, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=12, n_estimators=200, n_jobs=-1,
                      random_state=42, verbose=1)

In [18]:
# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\nModel performance on test set:\nMSE: {mse}\nR2 Score: {r2}")


Model performance on test set:
MSE: 4759.41595031795
R2 Score: 0.5035580777397277


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.1s finished


In [19]:
import os
import joblib

# Choose a specific directory
save_dir = r"/home/stormej/dev/varsha/data/machine_learning/models/random_forest"
os.makedirs(save_dir, exist_ok=True)

# Save model and scaler with full path
joblib.dump(model, os.path.join(save_dir, 'rainfall_prediction_model.joblib'))
joblib.dump(scaler, os.path.join(save_dir, 'rainfall_feature_scaler.joblib'))

['/home/stormej/dev/varsha/data/machine_learning/models/random_forest/rainfall_feature_scaler.joblib']

In [20]:
import os
import numpy as np
import rasterio
import joblib
from datetime import datetime
import re

def extract_date_from_filename(filepath):
    """
    Extract date from filename, handling various NDVI filename formats
    
    Parameters:
    - filepath: Full file path or filename
    
    Returns:
    - datetime object
    """
    # Extract just the filename from the full path
    filename = os.path.basename(filepath)
    
    # Try to extract date using regex patterns
    patterns = [
        r'(\d{4})(\d{2})(\w+)',  # MODIS_NDVI_Monthly_2017_06_June
        r'(\d{4})-(\d{2})',         # lst format 2017-06
    ]
    
    for pattern in patterns:
        match = re.search(pattern, filename)
        if match:
            if len(match.groups()) == 3:
                year, month, month_name = match.groups()
                try:
                    return datetime(int(year), int(month), 1)
                except ValueError:
                    continue
            elif len(match.groups()) == 2:
                year, month = match.groups()
                try:
                    return datetime(int(year), int(month), 1)
                except ValueError:
                    continue
    
    raise ValueError(f"Could not parse date from filename: {filename}")

def predict_spatial_rainfall(
    model, 
    scaler, 
    ndvi_files, 
    lst_files, 
    output_dir
):
    """
    Predict rainfall using trained model on spatial NDVI and LST data
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    for ndvi_file, lst_file in zip(ndvi_files, lst_files):
        # Extract date from filename
        try:
            file_date = extract_date_from_filename(ndvi_file)
        except ValueError as e:
            print(f"Skipping file due to date extraction error: {e}")
            continue
        
        # Load NDVI data
        with rasterio.open(ndvi_file) as ndvi_src:
            ndvi_data = ndvi_src.read(1)
            ndvi_flat = ndvi_data.flatten()
            
            # Get coordinates for each pixel
            shape = ndvi_data.shape
            coords = np.array([ndvi_src.xy(row, col) for row in range(shape[0]) for col in range(shape[1])])
            latitudes, longitudes = coords[:, 1], coords[:, 0]
        
        # Load LST data
        with rasterio.open(lst_file) as lst_src:
            lst_flat = lst_src.read(1).flatten()
        
        # Add month as cyclic feature
        month = file_date.month
        month_sin = np.sin(2 * np.pi * month / 12)
        month_cos = np.cos(2 * np.pi * month / 12)
        month_sin_array = np.full_like(ndvi_flat, month_sin)
        month_cos_array = np.full_like(ndvi_flat, month_cos)
        
        # Create feature stack
        feature_stack = np.column_stack([
            ndvi_flat,           # Normalized Difference Vegetation Index
            lst_flat,            # Land Surface Temperature
            latitudes,           # Latitude
            longitudes,          # Longitude
            month_sin_array,     # Cyclic month (sine)
            month_cos_array      # Cyclic month (cosine)
        ])
        
        # Remove NaN values
        valid_mask = (~np.isnan(feature_stack).any(axis=1))
        valid_features = feature_stack[valid_mask]
        
        # Scale features
        valid_features_scaled = scaler.transform(valid_features)
        
        # Predict rainfall
        predicted_rainfall = model.predict(valid_features_scaled)
        
        # Reconstruct spatial raster
        rainfall_raster = np.full(shape, np.nan)
        rainfall_raster.ravel()[valid_mask] = predicted_rainfall
        
        # Save predicted rainfall as GeoTIFF
        output_filename = f"predicted_rainfall_{file_date.strftime('%Y_%m')}.tif"
        output_filepath = os.path.join(output_dir, output_filename)
        
        with rasterio.open(ndvi_file) as src:
            profile = src.profile
            profile.update(dtype=rasterio.float32, count=1, compress='lzw')
        
        with rasterio.open(output_filepath, 'w', **profile) as dst:
            dst.write(rainfall_raster.astype(rasterio.float32), 1)
        
        print(f"Saved predicted rainfall for {file_date.strftime('%B %Y')}")

# Load model and scaler
model = joblib.load('/home/stormej/dev/varsha/data/machine_learning/models/random_forest/rainfall_prediction_model.joblib')
scaler = joblib.load('/home/stormej/dev/varsha/data/machine_learning/models/random_forest/rainfall_feature_scaler.joblib')

# Use raw string and escape backslashes for Windows paths
ndvi_files = [os.path.join(ndvi_dir, f) for f in os.listdir(ndvi_dir) if f.endswith('.tif')]

lst_files = [os.path.join(lst_dir, f) for f in os.listdir(lst_dir) if f.endswith('.tif')]

# Use os.path.join for output directory to avoid escape sequence issues
output_directory = os.path.join("/home/stormej/dev/varsha/data/machine_learning/random_forest", "prediction_tif_rf")

# Predict and save spatial rainfall
predict_spatial_rainfall(model, scaler, ndvi_files, lst_files, output_directory)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for January 2001


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for February 2001


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


...

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for October 2000


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for November 2000
Saved predicted rainfall for December 2000


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


In [23]:
#predicted rainfall at 1km
import os
import numpy as np
import rasterio
import joblib
from datetime import datetime
import re
import glob

def extract_date_from_filename(filepath):
    """
    Extract date from filename, handling various NDVI and LST filename formats.
    
    Parameters:
    - filepath: Full file path or filename.
    
    Returns:
    - datetime object.
    """
    # Extract just the filename from the full path
    filename = os.path.basename(filepath)
    
    # Try to extract date using regex patterns
    patterns = [
        r'modis_ndvi_avg_resampled_lst_(\d{4})-(\d{2})',  # NDVI files: modis_ndvi_avg_resampled_lst_2000-02.tif
        r'lst_monthly_mean_(\d{4})-(\d{2})',              # LST files: lst_monthly_mean_2000-02.tif
    ]
    
    for pattern in patterns:
        match = re.search(pattern, filename)
        if match:
            year, month = match.group(1), match.group(2)
            try:
                return datetime(int(year), int(month), 1)
            except ValueError:
                continue
    
    raise ValueError(f"Could not parse date from filename: {filename}")

def predict_spatial_rainfall(
    model, 
    scaler, 
    ndvi_dir, 
    lst_dir, 
    output_dir
):
    """
    Predict rainfall using trained model on spatial NDVI and LST data from directories
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get sorted lists of NDVI and LST files
    ndvi_files = sorted(glob.glob(os.path.join(ndvi_dir, '*.tif')))
    lst_files = sorted(glob.glob(os.path.join(lst_dir, '*.tif')))
    
    # Validate file lists
    if len(ndvi_files) != len(lst_files):
        raise ValueError("Number of NDVI and LST files do not match")
    
    for ndvi_file, lst_file in zip(ndvi_files, lst_files):
        # Extract date from filename
        try:
            file_date = extract_date_from_filename(ndvi_file)
        except ValueError as e:
            print(f"Skipping file due to date extraction error: {e}")
            continue
        
        # Load NDVI data
        with rasterio.open(ndvi_file) as ndvi_src:
            ndvi_data = ndvi_src.read(1)
            ndvi_flat = ndvi_data.flatten()
            
            # Get coordinates for each pixel
            shape = ndvi_data.shape
            coords = np.array([ndvi_src.xy(row, col) for row in range(shape[0]) for col in range(shape[1])])
            latitudes, longitudes = coords[:, 1], coords[:, 0]
        
        # Load LST data
        with rasterio.open(lst_file) as lst_src:
            lst_flat = lst_src.read(1).flatten()
        
        # Add month as cyclic feature
        month = file_date.month
        month_sin = np.sin(2 * np.pi * month / 12)
        month_cos = np.cos(2 * np.pi * month / 12)
        month_sin_array = np.full_like(ndvi_flat, month_sin)
        month_cos_array = np.full_like(ndvi_flat, month_cos)
        
        # Create feature stack
        feature_stack = np.column_stack([
            ndvi_flat,           # Normalized Difference Vegetation Index
            lst_flat,            # Land Surface Temperature
            latitudes,           # Latitude
            longitudes,          # Longitude
            month_sin_array,     # Cyclic month (sine)
            month_cos_array      # Cyclic month (cosine)
        ])
        
        # Remove NaN values
        valid_mask = (~np.isnan(feature_stack).any(axis=1))
        valid_features = feature_stack[valid_mask]
        
        # Scale features
        valid_features_scaled = scaler.transform(valid_features)
        
        # Predict rainfall
        predicted_rainfall = model.predict(valid_features_scaled)
        
        # Reconstruct spatial raster
        rainfall_raster = np.full(shape, np.nan)
        rainfall_raster.ravel()[valid_mask] = predicted_rainfall
        
        # Save predicted rainfall as GeoTIFF
        output_filename = f"downscaled_rainfall_{file_date.strftime('%Y_%m')}.tif"
        output_filepath = os.path.join(output_dir, output_filename)
        
        with rasterio.open(ndvi_file) as src:
            profile = src.profile
            profile.update(dtype=rasterio.float32, count=1, compress='lzw')
        
        with rasterio.open(output_filepath, 'w', **profile) as dst:
            dst.write(rainfall_raster.astype(rasterio.float32), 1)
        
        print(f"Saved predicted rainfall for {file_date.strftime('%B %Y')}")

# Load model and scaler
model = joblib.load('/home/stormej/dev/varsha/data/machine_learning/models/random_forest/rainfall_prediction_model.joblib')
scaler = joblib.load('/home/stormej/dev/varsha/data/machine_learning/models/random_forest/rainfall_feature_scaler.joblib')

# Use raw string and escape backslashes for Windows paths
ndvi_lst_resampled_dir = r"/home/stormej/dev/varsha/data/ndvi/ndvi_lst_resampled"
# ndvi_files = [os.path.join(ndvi_lst_resampled_dir, f) for f in os.listdir(ndvi_lst_resampled_dir) if f.endswith('.tif')]

lst_dir_monthly = r"/home/stormej/dev/varsha/data/lst/lst_tif_monthly"
# lst_files = [os.path.join(lst_dir_monthly, f) for f in os.listdir(lst_dir_monthly) if f.endswith('.tif')]

# Use os.path.join for output directory to avoid escape sequence issues
output_directory = r"/home/stormej/dev/varsha/data/machine_learning/random_forest/one_by_one_prediction_downscaled"

# Predict and save spatial rainfall
predict_spatial_rainfall(model, scaler, ndvi_lst_resampled_dir, lst_dir_monthly, output_directory)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for February 2000
Saved predicted rainfall for March 2000


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for April 2000
Saved predicted rainfall for May 2000


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


...

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for September 2022
Saved predicted rainfall for October 2022


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished


Saved predicted rainfall for November 2022
Saved predicted rainfall for December 2022


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
